# sentenceとwordのデータのまとめ

In [2]:
rosbag_dir = "/home/miyazawa/SIGVerse/sigros_ws/src/rosbag_database/src/bag/"

In [3]:
import sys
import glob
import os
import re
import numpy as np
import pandas as pd

sys.path.append("/opt/ros/indigo/lib/python2.7/dist-packages")
sys.path.append("/opt/tmc/ros/indigo/lib/python2.7/dist-packages")
sys.path.append("/home/miyazawa/SIGVerse/sigros_ws/devel/lib/python2.7/dist-packages")
sys.path.append("/home/miyazawa/catkin_ws/devel/lib/python2.7/dist-packages")
                
import rosbag
import rospy
from std_msgs.msg import  Header
from audio_module_msg.msg import AudioSentence

No handlers could be found for logger "rosout"


In [4]:
rosbag_path_list = glob.glob(os.path.join(rosbag_dir,"*/*"))
rosbag_path_list.sort()

* rosbag sentence to DataFrame  
データの書式をcaptionと合わせるように，secの精度でstampを取得

In [5]:
rosbag_df = pd.DataFrame(columns=["stamp","sentence"])
for rosbag_path in rosbag_path_list:
    bag = rosbag.Bag(rosbag_path)
    values = []
    for topic, msg, t in bag.read_messages('/AudioSentence'):
        values.append([int(msg.header.stamp.to_sec()), msg.sentences[0]])
    tmp_df = pd.DataFrame(data=values, columns=["stamp","sentence"])
    rosbag_df = rosbag_df.append(tmp_df, ignore_index=True)

* captionのデータを収集

In [6]:
caption_dir = glob.glob("/home/miyazawa/SIGVerse/data/0415/caption/exp*")
convert = lambda text: int(text) if text.isdigit() else text
alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ]
caption_dir.sort(key = alphanum_key)

In [7]:
values = []
action_nums = []
for exp_id, caption_path in enumerate(caption_dir):
    timestamp_path = glob.glob(caption_path + '/time*')
    timestamp_path.sort()
    for action_id, path in enumerate(timestamp_path):
        try:
            for stamp in np.loadtxt(path):
                values.append([int(stamp), exp_id, action_id])
        except:
            values.append([int(stamp), exp_id, action_id])
    action_nums.append(action_id)
caption_df = pd.DataFrame(data=values,columns=["stamp", "exp_num", "action_num"])

* spco の word　を収集

In [8]:
spco_word_path = glob.glob("/home/miyazawa/SIGVerse/sigros_ws/src/spco_formation/training_data/trial/word/*.txt")
spco_word_stamp_path = glob.glob("/home/miyazawa/SIGVerse/sigros_ws/src/spco_formation/training_data/trial/word/stamp/*")
convert = lambda text: int(text) if text.isdigit() else text
alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ]
spco_word_path.sort(key = alphanum_key)
spco_word_stamp_path.sort(key = alphanum_key)

In [9]:
values = []
for path,stamp_path in zip(spco_word_path,spco_word_stamp_path):
    index = int(path.split("/")[-1].strip(".txt"))
    word = np.loadtxt(path,dtype=str)
    stamp = int(np.loadtxt(stamp_path)) / pow(10,9)
    values.append([index,word,stamp])
df1 = pd.DataFrame(values,columns=["data_id","word","stamp"])

In [10]:
iteration_num = 10
values = []
count = 0
trial_num = 32
output_file = "/home/miyazawa/SIGVerse/sigros_ws/src/spco_formation/result/output_{0}.txt".format(trial_num)
for sen in np.loadtxt(output_file,dtype=str,delimiter="\n"):
    if "Iteration: {0}".format(iteration_num) in sen:
        class_id = int(sen.split()[-1])
        values.append([count,class_id])
        count += 1
df2 = pd.DataFrame(values,columns=["data_id","class_id"])

In [11]:
tmp = pd.merge(df1,df2,on="data_id",how="outer")
tmp = tmp.dropna()

* データのマージ

In [12]:
sentence_df = pd.merge(rosbag_df, caption_df, on="stamp", how="outer")
sentence_df = pd.merge(sentence_df, tmp , on="stamp", how="outer")
sentence_df = sentence_df.sort_values(by="stamp")
sentence_df = sentence_df.dropna()

In [13]:
caption_df = caption_df.sort_values(by=["stamp","exp_num","action_num"])
caption_df[caption_df.stamp.duplicated()]

,stamp,exp_num,action_num


In [14]:
place_hist = []
for exp_num, action_num_max in enumerate(action_nums):
    for action_num in range(action_num_max + 1):
        tmp_place_hist = np.zeros(100)
        tmp_df = sentence_df[(sentence_df.exp_num==exp_num) & (sentence_df.action_num==action_num)]
        for i in tmp_df.groupby(by="class_id").count().word.iteritems():
            class_id = int(i[0])
            num = i[1]
            tmp_place_hist[class_id] = num
        place_hist.append(tmp_place_hist)
place_hist = np.array(place_hist)

In [15]:
np.savetxt("/home/miyazawa/SIGVerse/sigros_ws/src/mmlda_bhmm/LearnData/place.Place",place_hist,fmt="%d")

In [16]:
sentence_df[sentence_df.word=="キッチン"]

,stamp,sentence,exp_num,action_num,data_id,word,class_id
24,1.522665e+09,キッチンの机に設置した,1.0,2.0,64.0,キッチン,5.0
53,1.522666e+09,キッチンの棚に設置,3.0,2.0,139.0,キッチン,84.0
86,1.522667e+09,キッチンの棚に設置,6.0,2.0,266.0,キッチン,82.0
87,1.522667e+09,キッチンの棚に設置した,6.0,2.0,267.0,キッチン,84.0
112,1.522667e+09,キッチンの棚に置いた,8.0,2.0,337.0,キッチン,84.0
138,1.522668e+09,キッチンの棚に置いた,10.0,2.0,413.0,キッチン,84.0
165,1.522669e+09,キッチンの棚に置いた,12.0,1.0,501.0,キッチン,84.0


In [17]:
sentence_df[sentence_df.word=="風呂"]

,stamp,sentence,exp_num,action_num,data_id,word,class_id
9,1.522665e+09,お風呂の棚に置く,0.0,1.0,28.0,風呂,97.0
10,1.522665e+09,お風呂の棚に置いた,0.0,1.0,29.0,風呂,97.0
36,1.522665e+09,お風呂の棚に置いた,2.0,1.0,96.0,風呂,97.0
64,1.522666e+09,お風呂の棚に設置する,4.0,1.0,176.0,風呂,97.0
66,1.522666e+09,お風呂の棚に,4.0,1.0,177.0,風呂,97.0
67,1.522666e+09,お風呂の棚に設置した,4.0,1.0,178.0,風呂,97.0
96,1.522667e+09,お風呂の棚に設置した,7.0,1.0,297.0,風呂,97.0
97,1.522667e+09,お風呂の棚に置いた,7.0,1.0,299.0,風呂,97.0
120,1.522668e+09,お風呂の棚に置く,9.0,1.0,368.0,風呂,97.0
123,1.522668e+09,お風呂の棚に置く,9.0,1.0,369.0,風呂,97.0


In [18]:
sentence_df[sentence_df.word=="寝室"]

,stamp,sentence,exp_num,action_num,data_id,word,class_id
2,1.522665e+09,寝室で飲み物を握る,0.0,0.0,3.0,寝室,5.0
4,1.522665e+09,寝室で飲み物を掴んだ,0.0,0.0,4.0,寝室,84.0
17,1.522665e+09,寝室で飲み物を把持する,1.0,1.0,46.0,寝室,5.0
18,1.522665e+09,寝室で飲み物を掴んだ,1.0,1.0,47.0,寝室,84.0
28,1.522665e+09,寝室で ぬいぐるみを把持する,2.0,0.0,69.0,寝室,84.0
29,1.522665e+09,寝室で ぬいぐるみを掴んだ,2.0,0.0,70.0,寝室,84.0
30,1.522665e+09,ここは 寝室だよ,2.0,0.0,71.0,寝室,84.0
47,1.522666e+09,寝室で ぬいぐるみをはじした,3.0,1.0,119.0,寝室,5.0
57,1.522666e+09,寝室で果物を握る,4.0,0.0,144.0,寝室,5.0
58,1.522666e+09,寝室で果物を把持する,4.0,0.0,145.0,寝室,5.0


In [19]:
sentence_df[sentence_df.word=="ドア前"]

,stamp,sentence,exp_num,action_num,data_id,word,class_id
14,1.522665e+09,ドア前でドアを押し開く,1.0,0.0,38.0,ドア前,5.0
40,1.522666e+09,ドア前でドアを押し開く,3.0,0.0,106.0,ドア前,71.0
41,1.522666e+09,ドア前でドアを開いた,3.0,0.0,107.0,ドア前,84.0
70,1.522667e+09,ドア前でドアを押す,5.0,0.0,188.0,ドア前,71.0
72,1.522667e+09,ドア前でドアを開ける,5.0,0.0,189.0,ドア前,84.0
79,1.522667e+09,ドア前でドアを押し開ける,6.0,0.0,214.0,ドア前,5.0
100,1.522667e+09,ドア前でドアを開く,8.0,0.0,308.0,ドア前,71.0
101,1.522667e+09,ドア前でドアを押す,8.0,0.0,309.0,ドア前,84.0
127,1.522668e+09,ドア前でドアを開く,10.0,0.0,379.0,ドア前,5.0
